In [23]:
import numpy as np
import pandas as pd

# loading the data from local file
data = pd.read_csv('dataset5.csv')
data.head()

,bpm,pace,time,distance,dist_week,dist_month,dist_quater,dist_year,count_week,count_month,...,cold_month,cold_quater,cold_yeqr,puls_week,puls_month,puls_quater,puls_year,gap,temperature,weight
0,149,357,4305,12050,24290,157020,372590,1050700,2,11,...,20,63,213,60,60,60,60,2,20,77.3
1,147,363,4403,12120,26080,152920,372480,1038580,2,11,...,21,65,214,60,60,60,60,4,21,77.3
2,149,366,4468,12170,32310,168800,372320,1049630,2,12,...,23,64,215,60,60,60,60,3,24,78.6
3,153,374,5205,13910,51940,154890,381430,1047120,3,11,...,23,64,215,61,60,60,60,2,29,76.9
4,151,356,6564,18400,49110,136490,363030,1044230,3,10,...,23,65,216,61,60,60,60,3,22,76.7


In [24]:
print(data.shape)
print(data.isna().sum().sum())

(270, 39)
0


In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270 entries, 0 to 269
Data columns (total 39 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   bpm            270 non-null    int64  
 1   pace           270 non-null    int64  
 2   time           270 non-null    int64  
 3   distance       270 non-null    int64  
 4   dist_week      270 non-null    int64  
 5   dist_month     270 non-null    int64  
 6   dist_quater    270 non-null    int64  
 7   dist_year      270 non-null    int64  
 8   count_week     270 non-null    int64  
 9   count_month    270 non-null    int64  
 10  count_quater   270 non-null    int64  
 11  count_year     270 non-null    int64  
 12  point_week     270 non-null    float64
 13  point_month    270 non-null    float64
 14  point_quater   270 non-null    float64
 15  point_year     270 non-null    float64
 16  steps_week     270 non-null    int64  
 17  steps_month    270 non-null    int64  
 18  steps_quat

In [26]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    data.drop('bpm', axis=1),
    data.bpm,
    test_size=0.15,
    random_state=42
)

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [27]:
X_train.mean(axis = 0)

array([0.28722123, 0.28163802, 0.27021875, 0.3585828 , 0.39590361,
       0.5601904 , 0.62171182, 0.38209607, 0.3706653 , 0.60779557,
       0.61382441, 0.63149345, 0.64554074, 0.62310485, 0.52856498,
       0.47342131, 0.67184701, 0.64834798, 0.79020964, 0.54203501,
       0.5125373 , 0.58768392, 0.66387789, 0.43405061, 0.4421872 ,
       0.44696436, 0.50295973, 0.64316906, 0.53729985, 0.49578552,
       0.71939272, 0.35851528, 0.37367436, 0.35633188, 0.51091703,
       0.03582482, 0.5748216 , 0.45943331])

In [28]:
from sklearn.ensemble import BaggingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# from sklearn.tree import DecisionTreeRegressor

# Базовая модель
lin = LinearRegression()

# Ансамбль из 10 деревьев
model = BaggingRegressor(
    estimator=lin,       # вместо base_estimator
    n_estimators=10,
    max_samples=0.85,     # каждая модель обучается на 85% случайных объектов
    max_features=1.0,    # все признаки
    random_state=111,
    n_jobs=-1
)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_train_pred = model.predict(X_train)

# print("Test MSE = %.4f" % mean_squared_error(y_test, y_pred))
# print("Train MSE = %.4f" % mean_squared_error(y_train, y_train_pred))
# print('-----')
print("Test RMSE = %.4f" % np.sqrt(mean_squared_error(y_test, y_pred)))
print("Train RMSE = %.4f" % np.sqrt(mean_squared_error(y_train, y_train_pred)))

Test RMSE = 4.1518
Train RMSE = 3.3336


In [29]:
# усредняем коэффициенты всех линейных регрессоров внутри бэггинга
coefs = np.mean([est.coef_ for est in model.estimators_], axis=0)

for i, coef in enumerate(coefs):
    print(f"Feature {i}: {coef:.4f}")

Feature 0: -28.1287
Feature 1: 21.8883
Feature 2: -11.3557
Feature 3: 3.8423
Feature 4: 11.6460
Feature 5: -41.5832
Feature 6: -52.2265
Feature 7: -9.5263
Feature 8: -11.1665
Feature 9: 29.9548
Feature 10: 43.3015
Feature 11: -6.1801
Feature 12: -0.0983
Feature 13: 19.8975
Feature 14: -11.9013
Feature 15: 2.9328
Feature 16: -1.1768
Feature 17: -3.1686
Feature 18: 9.5446
Feature 19: -4.3939
Feature 20: -0.1055
Feature 21: 13.9114
Feature 22: -15.1514
Feature 23: 3.5489
Feature 24: 16.5428
Feature 25: -2.6335
Feature 26: -7.1803
Feature 27: 1.0726
Feature 28: -0.7108
Feature 29: 1.5861
Feature 30: 0.2305
Feature 31: 0.2137
Feature 32: 0.2237
Feature 33: 2.2124
Feature 34: 4.1153
Feature 35: -4.1579
Feature 36: 6.9997
Feature 37: -10.0795
